In [1]:
# https://github.com/udacity/intro-to-ml-tensorflow/tree/master

In [ ]:
# %pip install tensorflow[and-cuda]
# %pip install tensorflow_datasets
# %pip install keras
# %pip install matplotlib
# %pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR) 
import keras
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

2025-05-22 00:33:24.670388: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 00:33:24.677486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747888404.685741  101973 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747888404.688111  101973 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747888404.694630  101973 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-1335.0162, shape=(), dtype=float32)


I0000 00:00:1747888405.694242  101973 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20409 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
print('Using:')
print('\t\u2022 TensorFlow version:', tf.__version__)
print('\t\u2022 tf.keras version:', keras.__version__)
print('\t\u2022 Running on GPU' if tf.test.is_gpu_available() else '\t\u2022 GPU device not found. Running on CPU')

Using:
	• TensorFlow version: 2.19.0
	• tf.keras version: 3.10.0
	• Running on GPU


I0000 00:00:1747888405.878027  101973 gpu_device.cc:2019] Created device /device:GPU:0 with 20409 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
# https://www.tensorflow.org/datasets/overview
iris_data, iris_info = tfds.load('iris', with_info=True, as_supervised=True)
print(iris_info)

tfds.core.DatasetInfo(
    name='iris',
    full_name='iris/2.1.0',
    description="""
    This is perhaps the best known database to be found in the pattern recognition
    literature. Fisher's paper is a classic in the field and is referenced
    frequently to this day. (See Duda & Hart, for example.) The data set contains
    3 classes of 50 instances each, where each class refers to a type of iris
    plant. One class is linearly separable from the other 2; the latter are NOT
    linearly separable from each other.
    """,
    homepage='https://archive.ics.uci.edu/ml/datasets/iris',
    data_dir='/home/oleg/tensorflow_datasets/iris/2.1.0',
    file_format=tfrecord,
    download_size=3.65 KiB,
    dataset_size=7.62 KiB,
    features=FeaturesDict({
        'features': Tensor(shape=(4,), dtype=float32),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('features', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
 

In [8]:
# Extract features from the iris training dataset
features = []
for feature, label in iris_data['train']:
    features.append(feature.numpy())
features = np.array(features)
features

2025-05-22 00:33:26.051396: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-05-22 00:33:26.058611: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


array([[5.1, 3.4, 1.5, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [5.7, 2.8, 4.5, 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [5.2, 3.4, 1.4, 0.2],
       [5.6, 2.9, 3.6, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5.5, 2.4, 3.7, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [7.7, 2.8, 6.7, 2. ],
       [7. , 3.2, 4.7, 1.4],
       [4.6, 3.2, 1.4, 0.2],
       [6.5, 3. , 5.2, 2. ],
       [5.5, 4.2, 1.4, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [4.8, 3. , 1.4, 0.1],
       [6.5, 3. , 5.8, 2.2],
       [7.6, 3. , 6.6, 2.1],
       [6.7, 3.3, 5.7, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [5.8, 4. , 1.2, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [5. , 3. , 1.6, 0.2],
       [6.9, 3.1, 5.1, 2.3],
       [6.1, 3. , 4.6, 1.4],
       [5.8, 2.7, 4.1, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [6.7, 3. , 5. , 1.7],
       [5.7, 2.6, 3.5, 1. ],
       [5.8, 2.8, 5.1, 2.4],
       [6.5, 3.2, 5.1, 2. ],
       [5.7, 3

In [9]:
# Get the label description from iris_info
label_description = iris_info.features['label'].names
print(label_description)

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


In [10]:
num_classes = iris_info.features['label'].num_classes
print('There are {:,} classes in our dataset'.format(num_classes))

num_training_examples = iris_info.splits['train'].num_examples
print('\nThere are {:,} rows in the training set'.format(num_training_examples))

There are 3 classes in our dataset

There are 150 rows in the training set


In [11]:
# creating pipelines 
# def normalize(image, label):
#     image = tf.cast(image, tf.float32)
#     image /= 255
#     return image, label

# batch_size = 64

# training_batches = training_set.cache().shuffle(num_training_examples//4).batch(batch_size).map(normalize).prefetch(1)

In [12]:
# https://www.tensorflow.org/guide/data_performance
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [13]:
# model = tf.keras.Sequential([
#         tf.keras.layers.Flatten(input_shape = (28,28,1)),
#         tf.keras.layers.Dense(256, activation = 'sigmoid'),
#         tf.keras.layers.Dense(10, activation = 'softmax')
# ])

# model.summary()

In [14]:
batch_size = 4

def preprocess(features, label):
    features = tf.cast(features, tf.float32)
    return features, label

train_ds = iris_data['train'] \
    .map(preprocess) \
    .cache() \
    .shuffle(num_training_examples) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

In [15]:
model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    keras.layers.Dense(12, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │           204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            39 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323 (1.26 KB)

 Trainable params: 323 (1.26 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# https://keras.io/api/layers/initializers/
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [17]:
weights_and_biases = model.get_weights()
len(weights_and_biases)

6

In [18]:
model.layers

[<Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>]

In [19]:
weights = model.get_layer(index=0).get_weights()[0]
weights

array([[-0.33674937, -0.09599638, -0.38478497,  0.4345426 , -0.46319216,
        -0.52352047, -0.06293416,  0.4535892 ,  0.26040035,  0.423122  ,
        -0.32109106,  0.3041814 , -0.42215186, -0.18200165,  0.24049401,
         0.20568979],
       [-0.37588832,  0.09499162, -0.00402063,  0.44492036,  0.29176164,
         0.403924  , -0.29701272,  0.47847283, -0.21506202, -0.23337337,
         0.12809801,  0.21251285, -0.13460553, -0.54506236,  0.15969759,
         0.18825537],
       [ 0.150729  , -0.43329513, -0.09634256, -0.03836381,  0.21158046,
        -0.05294922,  0.17219889,  0.12882209,  0.20019966, -0.5459466 ,
        -0.12725243, -0.3142112 ,  0.20560646, -0.1996679 ,  0.16854203,
        -0.06826982],
       [ 0.10371894, -0.14474747,  0.4210984 , -0.5214705 , -0.09199896,
         0.38360798, -0.00068259,  0.18499601, -0.18336773,  0.39163816,
        -0.36183167, -0.509046  ,  0.2887417 ,  0.04500872,  0.3323406 ,
        -0.32972312]], dtype=float32)

In [20]:
biases = model.get_layer(index=0).get_weights()[1]
biases

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [26]:
from keras import Model

# Subclassing 
class Network_1(Model):
    def __init__(self, num_classes = 2):
        super().__init__()
        self.num_classes = num_classes
    
        # Define layers 
        self.input_layer = keras.layers.Flatten()
        self.hidden_layer = keras.layers.Dense(256, activation = 'relu')
        self.output_layer = keras.layers.Dense(self.num_classes, activation = 'softmax')
    
    # Define forward Pass   
    def call(self, input_tensor):
        x = self.input_layer(input_tensor)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
    
        return x

In [ ]:
# Create a model object
subclassed_model = Network_1(10)

# Build the model, i.e. initialize the model's weights and biases
subclassed_model.build((None, 28, 28, 1))

subclassed_model.summary()


Model: "network_1_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# adding layers
model = keras.Sequential()

model.add(keras.layers.Flatten(input_shape = (28,28,1)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,450 (99.41 KB)

 Trainable params: 25,450 (99.41 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
layer_neurons = [512, 256, 128, 56, 28, 14]

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(keras.layers.Dense(neurons, activation='relu'))
            
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary() 

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 56)             │         7,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 28)             │         1,596 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 14)             │           406 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           150 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 575,520 (2.20 MB)

 Trainable params: 575,520 (2.20 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# clearing the graph
keras.backend.clear_session()

layer_neurons = [512, 256, 128, 56, 28, 14]

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28,28,1)))

for neurons in layer_neurons:
    model.add(keras.layers.Dense(neurons, activation='relu'))
            
model.add(keras.layers.Dense(10, activation='softmax'))
          
model.summary() 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 56)             │         7,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 28)             │         1,596 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 14)             │           406 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           150 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 575,520 (2.20 MB)

 Trainable params: 575,520 (2.20 MB)

 Non-trainable params: 0 (0.00 B)